In [1]:
url = "https://www.joc.or.jp/news/index.html?p={}"

In [2]:
import requests
from bs4 import BeautifulSoup

In [5]:
r = requests.get(url.format(1))
soup = BeautifulSoup(r.content, "html.parser")

section = soup.find("div", {"id": "news_body"})
links = section.find_all("li", {"class": "clearfix"})

In [14]:
rows = [[l.find("div", {"class": "ttl"}).find("a"), l.find("div", {"class": "date"}).get_text(), l.find("div", {"class": "category"}).get_text()] for l in links]
rows = [r for r in rows if r[0] is not None]
urls = ["https://www.joc.or.jp/news/" + r[0]['href'] for r in rows]
assert len(urls) == len(rows)
rows = [[url, r[0].get_text(), r[1], r[2]] for r, url in zip(rows, urls)]
rows[:3]

[['https://www.joc.or.jp/news/./detail.html?id=10412',
  '織田幹雄さん快挙から９０年 努力、工夫で五輪金１号',
  '2018.08.03',
  'オリンピック'],
 ['https://www.joc.or.jp/news/./detail.html?id=10411',
  '世界バド、高橋礼・松友組敗れる 山口、奥原、桃田ら８強入り',
  '2018.08.02',
  'その他競技情報'],
 ['https://www.joc.or.jp/news/./detail.html?id=10410',
  'ソフトボール、日本が伊に快勝 世界選手権開幕',
  '2018.08.02',
  'その他競技情報']]

In [3]:
def get_rows(url, page):
    r = requests.get(url.format(page))
    soup = BeautifulSoup(r.content, "html.parser")
    section = soup.find("div", {"id": "news_body"})
    links = section.find_all("li", {"class": "clearfix"})
    rows = [[l.find("div", {"class": "ttl"}).find("a"), l.find("div", {"class": "date"}).get_text(), l.find("div", {"class": "category"}).get_text()] for l in links]
    rows = [r for r in rows if r[0] is not None]
    urls = ["https://www.joc.or.jp/news/" + r[0]['href'] for r in rows]
    assert len(urls) == len(rows)
    rows = [[url, r[0].get_text(), r[1], r[2]] for r, url in zip(rows, urls)]
    return rows

In [4]:
with open("news_joc.tsv", "w") as f:
    f.write("{}\t{}\t{}\t{}\t\n".format("url","title","date","category"))

import pymp
with pymp.Parallel(100) as p:
    for page in p.range(1, 444):
        try:
            rows = get_rows(url, page)
            for row in rows:
                with open("news_joc.tsv", "a") as f:
                    f.write("{}\t{}\t{}\t{}\t\n".format(*row))
            print("DONE:{}".format(page))
        except:
            print("ERROR:{}".format(page))

DONE:41
DONE:46
DONE:6
DONE:21
DONE:36
DONE:66
DONE:61
DONE:16
DONE:81
DONE:1
DONE:26
DONE:136
DONE:91
DONE:56
DONE:126
DONE:116
DONE:86
DONE:121
DONE:201
DONE:51
DONE:141
DONE:146
DONE:181
DONE:11
DONE:186
DONE:260
DONE:76
DONE:71
DONE:31
DONE:96
DONE:111
DONE:191
DONE:288
DONE:272
DONE:166
DONE:176
DONE:196
DONE:228
DONE:220
DONE:324
DONE:268
DONE:248
DONE:232
DONE:106
DONE:252
DONE:161
DONE:256
DONE:151
DONE:240
DONE:280
DONE:236
DONE:340
DONE:308
DONE:396
DONE:224
DONE:368
DONE:284
DONE:244
DONE:440
DONE:206
DONE:328
DONE:436
DONE:300
DONE:292
DONE:216
DONE:304
DONE:211
DONE:276
DONE:101
DONE:171
DONE:336
DONE:156
DONE:131
DONE:428
DONE:384
DONE:296
DONE:264
DONE:364
DONE:356
DONE:404
DONE:360
DONE:352
DONE:432
DONE:312
DONE:376
DONE:372
DONE:316
DONE:332
DONE:344
DONE:348
DONE:388
DONE:380
DONE:424
DONE:408
DONE:412
DONE:420
DONE:320
DONE:416
DONE:42
DONE:392
DONE:22
DONE:47
DONE:400
DONE:7
DONE:2
DONE:17
DONE:202
DONE:12
DONE:57
DONE:37
DONE:82
DONE:127
DONE:197
DONE:192
DONE:62


In [6]:
import pandas as pd
df = pd.read_csv("news_joc.tsv", sep="\t")
df = df[['url', 'title', 'date', 'category']]

In [9]:
r = requests.get(df['url'].iloc[0])
soup = BeautifulSoup(r.content, "html.parser")

section = soup.find("div", {"class": "news_box"})
body = section.find("p").get_text()
body.replace("\r","").replace("\n","").replace("\u3000", "")

'【ドバイ共同】バドミントンのスーパーシリーズ（ＳＳ）ファイナルは１４日、ドバイで１次リーグ第２戦が行われ、女子ダブルスで世界選手権銀メダルの福島由紀、広田彩花組はリオデジャネイロ五輪銅メダルの鄭径恩、申昇チャン組（韓国）を２―０で下し、２連勝となった。米元小春、田中志穂組は中国ペアにストレート勝ちして通算１勝１敗。女子シングルスの山口茜は何冰嬌（中国）に２―１で競り勝ち、２戦全勝とした。男子ダブルスの保木卓朗、小林優吾組は中国ペアに０―２で敗れ、混合ダブルスの数野健太、栗原文音組は香港ペアに１―２で屈して、ともに２連敗となった。'

In [11]:
def get_content(url):
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        section = soup.find("div", {"class": "news_box"})
        body = section.find("p").get_text()
        return body.replace("\r","").replace("\n","").replace("\u3000", "")
    except:
        return ""

In [ ]:
urls = df['url'].tolist()

with open("news_joc_content.tsv", "w") as f:
    f.write("{}\t{}\n".format("id", "content"))

with pymp.Parallel(100) as p:
    for index in p.range(0, len(urls)):
        try:
            content = get_content(urls[index])
            with open("news_joc_content.tsv", "a") as f:
                f.write("{}\t{}\n".format(index, content))
            if index % 100 == 0:
                print("DONE:{}".format(index))
        except:
            print("ERROR:{}".format(index))

DONE:0
DONE:6300
DONE:8500
DONE:4100
DONE:7800
DONE:5600
DONE:3300
DONE:2500
DONE:100
DONE:1700
DONE:4900
DONE:900
DONE:7100
DONE:8600
DONE:6400
DONE:4200
DONE:5700
DONE:7900
DONE:3400
DONE:2600
DONE:1800
DONE:7200
DONE:5000
DONE:1000
DONE:200
DONE:8700
DONE:4300
DONE:6500
DONE:8000
DONE:3500
DONE:2700
DONE:5800
DONE:1900
DONE:1100
DONE:7300
DONE:300
DONE:5100
DONE:8800
DONE:6600
DONE:4400
DONE:8100
DONE:2800
DONE:3600
DONE:2000
DONE:400
DONE:5900
DONE:1200
DONE:5200
DONE:7400
DONE:6700
DONE:2100
DONE:4500
DONE:3700
DONE:2900
DONE:500
DONE:1300
DONE:8200
DONE:6000
DONE:7500
DONE:5300
DONE:2200
DONE:3000
DONE:6800
DONE:3800
DONE:4600
